# Beopest example for PEST-DSM2 Calibration
This notebook sets up a beopest master on a node and uses the hostname for the beopest master to start beopest slaves.
These are then used for submitting a sample .pst and model for an optimization run

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [119]:
import dmsbatch
from dmsbatch import create_batch_client, create_blob_client
import datetime

## First create a batch client from the config file

In [27]:
client = create_batch_client('../tests/data/dmsbatch.config')
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

In [86]:
app_pkgs = [('dsm2', '8.2.c5aacef7', 'DSM2-8.2.c5aacef7-win32/bin'),
            ('vista', '1.0-v2019-05-28', 'vista/bin'),
            ('python', 'testpest1', ''),
            ('unzip', '5.51-1', 'bin'),
            ('pest','17.3','')]

### Show vms available

https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series

In [5]:
#display(client.skus_available())

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

In [42]:
pool_name='pestpool'

In [43]:
# client.create_pool(pool_name,
#                    1,
#                    app_packages=[(app,version) for app,version,_ in app_pkgs], 
#                    vm_size='standard_f2s_v2', 
#                    tasks_per_vm=1, # keeping only 1 task per vm due to issue with HEC dss vue conflicting with catalogs!
#                    os_image_data=('microsoftwindowsserver', 'windowsserver', '2019-datacenter-core')
#                   )

AttributeError: module 'azure.batch.models' has no attribute 'AccountListSupportedImagesOptions'

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail which is ok as its already there

In [32]:
job_name='pestdsm2'
# copy_common_task = client.create_task_copy_file_to_shared_dir(job_name,'common.zip',file_path='.')
# client.create_job(job_name,pool_name,prep_task=copy_common_task)

BatchErrorException: {'additional_properties': {}, 'lang': 'en-US', 'value': 'The specified job already exists.\nRequestId:3688d543-c149-4f1f-8fe6-532e818be883\nTime:2022-02-14T06:42:15.8852384Z'}

## Upload the directory to blob storage
Upload the directory to a blob container with same name as the job

In [ ]:
# local_dir = '../tests/data/PEST_DSM2'
# input_file=blob_client.zip_and_upload(job_name,'',local_dir,30)
# #
# input_file = client.create_input_file_spec(job_name,blob_prefix=input_file.file_path,file_path='.')

if job folder already uploaded onto blob

In [122]:
# input_file = client.create_input_file_spec(job_name,blob_prefix='PEST_DSM2.zip',file_path='.')
input_file = client.create_input_file_spec(job_name,blob_prefix='PEST_DSM2_small.zip',file_path='.')

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

PEST for small calib

In [123]:
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'beopest_master_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path('echo hostname %COMPUTERNAME% & hostname '
                                          +'& call %AZ_BATCH_APP_PACKAGE_python#testpest1%/scripts/activate.bat '
                                          +'& unzip PEST_DSM2_small.zip & call runPESTsetup.bat & call runPEST.bat',
                                           app_pkgs)
print(task_name)
print(cmd_string)

beopest_master_1645034511
cmd /c set "PATH=%AZ_BATCH_APP_PACKAGE_dsm2#8.2.c5aacef7%/DSM2-8.2.c5aacef7-win32/bin;%AZ_BATCH_APP_PACKAGE_vista#1.0-v2019-05-28%/vista/bin;%AZ_BATCH_APP_PACKAGE_python#testpest1%/;%AZ_BATCH_APP_PACKAGE_unzip#5.51-1%/bin;%AZ_BATCH_APP_PACKAGE_pest#17.3%/;%PATH%" & echo hostname %COMPUTERNAME% & hostname & call %AZ_BATCH_APP_PACKAGE_python#testpest1%/scripts/activate.bat & unzip PEST_DSM2_small.zip & call runPESTsetup.bat & call runPEST.bat


beoPEST for big calib

In [112]:
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'beopest_master_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path('echo hostname %COMPUTERNAME% & hostname '
                                          +'& call %AZ_BATCH_APP_PACKAGE_python#testpest1%/scripts/activate.bat '
                                          +'& unzip PEST_DSM2.zip & call runPESTsetup.bat & call start_pest_master.bat',
                                           app_pkgs)
print(task_name)
print(cmd_string)

beopest_master_1644986515
cmd /c set "PATH=%AZ_BATCH_APP_PACKAGE_dsm2#8.2.c5aacef7%/DSM2-8.2.c5aacef7-win32/bin;%AZ_BATCH_APP_PACKAGE_vista#1.0-v2019-05-28%/vista/bin;%AZ_BATCH_APP_PACKAGE_python#testpest1%/;%AZ_BATCH_APP_PACKAGE_unzip#5.51-1%/bin;%AZ_BATCH_APP_PACKAGE_pest#17.3%/;%PATH%" & echo hostname %COMPUTERNAME% & hostname & call %AZ_BATCH_APP_PACKAGE_python#testpest1%/scripts/activate.bat & unzip PEST_DSM2.zip & call runPESTsetup.bat & call start_pest_master.bat


In [124]:
permissions = dmsbatch.commands.azureblob.BlobPermissions.WRITE
output_dir_sas_url = blob_client.get_container_sas_url('pestdsm2', permissions)
print(output_dir_sas_url)

https://dwrmodelingstore.blob.core.windows.net/pestdsm2?se=2022-02-16T20%3A02%3A05Z&sp=w&sv=2018-03-28&sr=c&sig=WCFS8nXGPWiIo6ddmWmtnzgbr9/qzAdzFYAexGlkJhU%3D


In [125]:
std_out = client.create_output_file_spec(
    '../std*.txt', output_dir_sas_url, blob_path='output')
pest_out = client.create_output_file_spec(
    '**/*.rec', output_dir_sas_url, blob_path='output')
stat_out = client.create_output_file_spec(
    '**/metrics.dat', output_dir_sas_url, blob_path='output')
dsm2_out = client.create_output_file_spec(
    '**/hist_v82_CALIB.dss', output_dir_sas_url, blob_path='output')

In [126]:
pest_task = client.create_task(task_name,cmd_string,
                               resource_files=[input_file],
                               output_files=[std_out, pest_out,stat_out,dsm2_out])

### Next submit the task for master and wait for hostname to show up in the stdout.txt

In [127]:
client.submit_tasks(job_name,[pest_task])

In [105]:
def parse_hostname(client, job_id, task_id):
    stdout=client.read_task_file_as_string(job_id,task_id,'stdout.txt')
    import io
    fh=io.StringIO(stdout)
    line=fh.readline()
    hostname = line.split('hostname')[1].strip()
    return hostname

In [115]:
import time
while True:
    try:
        hostname = parse_hostname(client, 'pestdsm2',pest_task.id)
        break;
    except:
        time.sleep(10)
        continue

In [116]:
print('Beopest master is running on',hostname)

Beopest master is running on a20b29581000001


## Next start the slaves (workers) with the hostname information

In [1]:
task_name = f'beopest_slave_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path('echo hostname %COMPUTERNAME% & hostname '
                                          +'& call %AZ_BATCH_APP_PACKAGE_python#testpest1%/scripts/activate.bat '
                                          +'& unzip PEST_DSM2.zip & call runPESTsetup.bat & call start_pest_slave.bat',
                                           app_pkgs)
print(task_name)
print(cmd_string)

NameError: name 'tsnow' is not defined

In [118]:
pest_slave_tasks = [client.create_task(task_name+'_%d'%i,
                                       cmd_string,
                                       resource_files=[input_file],
                                       env_settings={'MASTER_HOSTNAME':hostname}) for i in range(2,7)]
client.submit_tasks(job_name,pest_slave_tasks)

## Finally resize the pool to 0 to save costs

In [ ]:
# client.resize_pool(pool_name,0)